In [2]:
import cv2
import tensorflow as tf
import numpy as np

# Load the saved model
model = tf.keras.models.load_model('facemask_model.h5')

# Define the face detection algorithm
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Define a function to preprocess the image for model input
def preprocess_image(img):
    img = cv2.resize(img, (96, 96))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# Define a function to draw the predicted label on the image
def draw_label(img, text, pos, bg_color):
    font_face = cv2.FONT_HERSHEY_SIMPLEX
    scale = 1.0
    color = (255, 255, 255)
    thickness = cv2.FILLED
    margin = 10

    # Get the size of the text box
    (text_width, text_height), baseline = cv2.getTextSize(text, font_face, scale, thickness)

    # Background rectangle
    rect_end = (pos[0] + text_width + margin, pos[1] + text_height + margin)
    cv2.rectangle(img, pos, rect_end, bg_color, thickness)

    # Text label
    cv2.putText(img, text, (pos[0]+margin//2, pos[1]+text_height+margin//2),
                font_face, scale, color, 1, cv2.LINE_AA)

# Access the webcam
cap = cv2.VideoCapture(0)

# Loop over the video frames
while True:
    # Read a frame from the video stream
    ret, frame = cap.read()

    # Apply the face detection algorithm to get the face regions
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    # Loop over the face regions and make predictions for each one
    for (x,y,w,h) in faces:
        # Extract the face region from the frame
        face_img = frame[y:y+h, x:x+w]

        # Preprocess the face image for model input
        input_img = preprocess_image(face_img)

        # Get the predicted probabilities for the two classes
        probs = model.predict(input_img)[0]

        # Get the predicted label and draw it on the image
        label = "No Mask" if probs[1] > probs[0] else "Mask"
        color = (0, 0, 255) if label == "No Mask" else (0, 255, 0)
        draw_label(frame, label, (x, y), color)

        # Draw a rectangle around the face region
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)

    # Show the frame with the predicted labels and rectangles
    cv2.imshow('Face Mask Detector', frame)

    # Check if the user pressed the 'q' key to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close the window
cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 25ms/step


1/1 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 26ms/step
